# Tutorial 1. Basic DNA sequence manipulation

© 2018 Manuel Razo. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT)

---

In [45]:
# Import relevant libraries
import numpy as np # For numerical computation
import pandas as pd # For data manipulation

# Import bioinformatic tools
import Bio.AlignIO # To read sequence alignments
import Bio.Seq # Tools to manipulate sequences

# Import plotting utilities
import matplotlib.pyplot as plt
import seaborn as sns

# Magic function to make matplotlib inline; other style specs must come AFTER
%matplotlib inline

# Set plotting style
sns.set_style('darkgrid')
sns.set_context('notebook')

# This enables SVG graphics inline
%config InlineBackend.figure_format = 'svg'

# Consider the Leviathan's teeth.

As we saw in lecture during the WoW (Week of Whales), the evolution of Cetaceans has served as an incredible puzzle for evolutionary biologists starting from Darwin himself. The transition from limbs-to-fins is one of the best documented cases within the fossil record. But in the age of modern DNA sequencing the cross-talking between classic paleontology and DNA science is still a rare event.

In the beautiful [paper by Meredith et al.](http://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1000634) they exploit this interaction between the fossil record and the molecular evidence to "*provide manifest evidence for the predictive power of Darwin's theory*." In this work they are interested in the conservation of teeth among placental mammals. In particular they study the enamelin (*ENAM*) gene as a 'molecular fossil' that they can compare directly with morphological features well preserved in the fossil record.

The *ENAM* gene participates in the production and secretion of enamel, the hardest substance found in vertebrates that forms the outer cap of teeth. Being such a hard compound gives is a rich representation in the fossil record of placental mammals. But there are group of toothless mammals (edentulous) such as [pangolins](https://en.wikipedia.org/wiki/Pangolin), [baleen whales](https://en.wikipedia.org/wiki/Baleen_whale), and [anteaters](https://en.wikipedia.org/wiki/Anteater), and mammals with enamelless teeth such as [sloths](https://en.wikipedia.org/wiki/Sloth), [armadillos](https://en.wikipedia.org/wiki/Armadillo), [sperm whales](https://en.wikipedia.org/wiki/Sperm_whale), and [aardvarks](https://en.wikipedia.org/wiki/Aardvark).

If all of these "outlier" groups of mammals descended from a common ancestor that had enamel coated teeth, Darwin's theory makes the simple prediction that the *ENAM* gene should be present in the genome of all these species, but it must be pseudogenized. What this means is that if one were to sequence the gene of edentulous and enamelless mammals and compare it with other placental mammals, there should be features such as stop codons, insertions or frame-shifts that would eliminate the functionality of the gene.

Read the data.

In [2]:
# Define data directory
datadir = './data/'

# Read alignment file
enamel_aln = Bio.AlignIO.read(datadir + 'enamel_alignment.nxs', 'nexus')

# Print number of sequences in alignment
print('There are {:d} sequences in the alignment'.format(len(enamel_aln)))

There are 49 sequences in the alignment


Extract sequence and species name from alignment object.

In [3]:
# Extract sequences and names for each entry in alignment
enamel_seq = [record.seq for record in enamel_aln]
enamel_name = [record.name for record in enamel_aln]

Remove non-DNA characters.

In [4]:
# Let's remove the missing "?" and gap "-" characters from all records
enamel_clean_seq = [seq.ungap('?') for seq in enamel_seq]
enamel_clean_seq = [seq.ungap('-') for seq in enamel_clean_seq]

Generate DataFrame saving all 3 ORF translations.

In [16]:
# Initialize matrix to save 3 ORF translations
columns = ['DNA', 'protein', 'frame', 'name']
enamel_prot = pd.DataFrame(columns=[])
for i, seq in enumerate(enamel_clean_seq):
    for frame in range(3):
        # Initialize array to save info
        species = pd.Series([enamel_name[i], str(seq)], index=['name', 'DNA'])
        length = 3 * (len(seq) - frame // 3)
        prot_seq = seq[frame: frame + length].translate()
        # Append this ORF to pd.Series
        species = species.append(pd.Series([str(prot_seq), frame],
                                           index=['protein', 'frame']))
        enamel_prot = enamel_prot.append(species, ignore_index=True)

# Print the head of the table
enamel_prot.head()

/Users/razo/anaconda3/lib/python3.6/site-packages/Bio/Seq.py:2095: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


,DNA,frame,name,protein
0,GACACCCGTATTATTCAGAAGAGATGTATGAACAAGATTATGAACA...,0.0,Vombatus_ursinus,DTRIIQKRCMNKIMNSPKRRIHPKWRAPPQPLPQTPQPLRTIQLNQ...
1,GACACCCGTATTATTCAGAAGAGATGTATGAACAAGATTATGAACA...,1.0,Vombatus_ursinus,TPVLFRRDV*TRL*TAQRGGSTQSGEHHLSPSPKHHSP*EQFNSTN...
2,GACACCCGTATTATTCAGAAGAGATGTATGAACAAGATTATGAACA...,2.0,Vombatus_ursinus,HPYYSEEMYEQDYEQPKEEDPPKVESTTSAPPPNTTALENNSTQPT...
3,GACCCCCATATTATTCAGAAGAGATGTTTGAACAAGACTTTGAAAA...,0.0,Monodelphis_domestica,DPHIIQKRCLNKTLKSPKRKIPPK*RVPPQPLQRTPRFLRPIQLKQ...
4,GACCCCCATATTATTCAGAAGAGATGTTTGAACAAGACTTTGAAAA...,1.0,Monodelphis_domestica,TPILFRRDV*TRL*KAQRGRSPQSREYHRSPSNELHGS*DQFNSSN...


Find how many stop codons per ORF.

In [17]:
# Initialize array to save open reading frames
stop_codons = np.zeros(len(enamel_prot))

# Loop through protein sequences counting stop codons
for i, prot in enumerate(enamel_prot.protein):
    stop_codons[i] = prot.count('*')

# Append number of stop codons as column to the dataframe
enamel_prot['num_stops'] = stop_codons

# Print head of expanded DataFrame
enamel_prot[['name', 'frame', 'num_stops']].head()

,name,frame,num_stops
0,Vombatus_ursinus,0.0,23.0
1,Vombatus_ursinus,1.0,71.0
2,Vombatus_ursinus,2.0,0.0
3,Monodelphis_domestica,0.0,31.0
4,Monodelphis_domestica,1.0,72.0


In [39]:
# Initialize a DataFrame to save per species whether or not all 
# ORFs have stop codons
df_stops = pd.DataFrame(columns=['name', 'all_stops'])

# Loop through species and ask if all frames have a stop codon
for i, species in enumerate(enamel_name):
    # Extract the data
    stops = enamel_prot[enamel_prot.name == species].num_stops
    # Ask if all ORFs have stop codons
    species_stop = np.all(stops > 0)
    # Append result to DataFrame
    df_stops = df_stops.append(pd.Series([species, species_stop],
                                            index=['name', 'all_stops']),
                                 ignore_index=True)

# Print header of DataFrame
df_stops.head()

,name,all_stops
0,Vombatus_ursinus,False
1,Monodelphis_domestica,False
2,Eubalaena_glacialis,True
3,Eubalaena_australis,True
4,Megaptera_novaeangliae,True


In [44]:
len(df_stops[df_stops.all_stops == True])

20